In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
import seaborn as sns
import requests

In [ ]:
# Formatting DataFrames
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
deputados = pd.read_csv('deputados.csv', delimiter=';')

In [ ]:
deputados

In [ ]:
# Try to find your representative's name changing the below variable
part_of_name = 'name here'
deputados[deputados['nomeCivil'].str.contains(part_of_name, case=False)]

In [ ]:
# A workaround to get the representative unique ID: last part of the URI
list(deputados[deputados['nomeCivil'].str.contains(part_of_name, case=False)]['uri'])

In [ ]:
# import annual budget, then compile
budget_2019 = pd.read_csv('Ano-2019.csv', delimiter=';')
budget_2020 = pd.read_csv('Ano-2020.csv', delimiter=';')
budget_2021 = pd.read_csv('Ano-2021.csv', delimiter=';')
budget_2022 = pd.read_csv('Ano-2022.csv', delimiter=';')

In [ ]:
budget_2019

In [ ]:
budget_2019.columns

In [ ]:
# Ugly but working way of unifying the database...
# Insert representative UID in the below variable
representative_uid: int = 1234567890
budget = budget_2019[budget_2019['ideCadastro'] == representative_uid].copy()
budget = pd.concat([budget, budget_2020[budget_2020['ideCadastro'] == representative_uid].copy()])
budget = pd.concat([budget, budget_2021[budget_2021['ideCadastro'] == representative_uid].copy()])
budget = pd.concat([budget, budget_2022[budget_2022['ideCadastro'] == representative_uid].copy()])

In [ ]:
# View all columns of dataframe
original_num_columns = pd.options.display.max_columns
pd.options.display.max_columns = len(budget.columns)
budget.head(7)

In [ ]:
# Revert settings
pd.options.display.max_columns = original_num_columns

In [ ]:
# Create a datetime column
budget['date'] = pd.to_datetime(budget['numAno'].astype(str) + "/" + budget['numMes'].astype(str) + "/01")

In [ ]:
budget_grouped = budget.groupby(['numAno', 'numMes', 'txtDescricao'])['vlrLiquido'].sum().reset_index().copy()

In [ ]:
# Creating columns with a proper date format
budget_grouped['date'] = pd.to_datetime(
    budget_grouped['numAno'].astype(str)
    + "/" + budget_grouped['numMes'].astype(str) 
    + "/01"
)

In [ ]:
budget_grouped

In [ ]:
# I'm going to reclassify the expenses with less categories, for clarity
descr_reclass = {
    'MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR': 'Office Expenses',
    'COMBUSTÍVEIS E LUBRIFICANTES.': 'Transportation',
    'CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.': 'Consulting',
    'DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.': 'Publicity',
    'PASSAGEM AÉREA - REEMBOLSO': 'Air Travel',
    'TELEFONIA': 'Miscelaneous',
    'SERVIÇOS POSTAIS': 'Miscelaneous',
    'ASSINATURA DE PUBLICAÇÕES': 'Miscelaneous',
    'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR': 'Catering',
    'HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.': 'Lodging',
    'LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES': 'Transportation',
    'SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO': 'Transportation',
    'PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS': 'Transportation',
    'PASSAGEM AÉREA - SIGEPA': 'Air Travel',
    'PASSAGEM AÉREA - RPA': 'Air Travel',
    'SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.': 'Security Services',
    'PARTICIPAÇÃO EM CURSO, PALESTRA OU EVENTO SIMILAR': 'Courses',
    'LOCAÇÃO OU FRETAMENTO DE AERONAVES': 'Chartered Flights',
    'LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES': 'Vessel/Boat Rental'
}

budget['txtClassification'] = budget['txtDescricao'].map(descr_reclass).astype('category')

budget['txtClassification'].unique()

In [ ]:
# Checking new categories / grouping
budget_grouped = budget.groupby(['numAno', 'numMes', 'txtClassification'])['vlrLiquido'].sum().reset_index().copy()
budget_grouped.reset_index()
budget_grouped['date'] = pd.to_datetime(
    budget_grouped['numAno'].astype(str)
    + "/" + budget_grouped['numMes'].astype(str) 
    + "/01"
)
budget_grouped

In [ ]:
# PLOT OF SELECTED REPRESENTATIVE'S EXPENSES
# List of years in legislature
years = list(budget_grouped['numAno'].unique())

# List of categories of expenses
categ = list(budget_grouped['txtClassification'].unique().sort_values())

# SUBPLOTS
fig, ax = plt.subplots(figsize=(16, 3 * len(years)), sharex=False, sharey=True, ncols=1, nrows=len(years))

# CONSTANTS
bar_width = 0.6 / len(categ)
bar_xlim = (0, 13)
bar_y_interval = 2500
bar_ylim = (0, (max(budget_grouped['vlrLiquido']) // bar_y_interval + 1) * bar_y_interval)
bar_xticks_major = range(1, 13)
bar_xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# CREATE A BAR PLOT FOR EACH YEAR
for _i, _year in enumerate(years):
    
    # CREATE SHADING TO SEPARATE MONTHS
    for _k, _month in enumerate(bar_xlabels):
        if _k % 2 != 0:
            ax[_i].axvspan(
                xmin=(_k - 0.5),
                xmax=(_k + 0.5),
                ymin=0,
                ymax=1,
                color='lightgray',
                alpha=0.35
            )
    
    # CREATE DATA AND BARS FOR EACH CATEGORY
    for _j, _categ in enumerate(categ):
        bar_data = (
            budget_grouped[
                (budget_grouped['numAno'] == _year)
                & (budget_grouped['txtClassification'] == _categ)
            ].groupby(['numMes']).sum()
        )
        
        _x = bar_data.index - (len(categ) / 2 - _j) * bar_width
        _y = bar_data['vlrLiquido']
        ax[_i].bar(x=_x, height=_y, width=bar_width, align='center', color=sns.color_palette('tab20')[_j])
    
    # LIMITS
    ax[_i].set_xlim(bar_xlim)
    ax[_i].set_ylim(bar_ylim)
    
    # TICK LABELS - REMOVE ALL FROM X AXIS
    ax[_i].set_xticks([])
    
    # AXIS LABELS
    ax[_i].set_ylabel("BRL 1k", labelpad=10)
    
    # SECONDARY Y AXIS
    # The creation of a secondary axis requires transformation functions. I just need the label...
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.secondary_yaxis.html
    sec_y = ax[_i].secondary_yaxis(location='right', functions=(lambda t: t, lambda t: t))
    sec_y.set_yticks([])
    sec_y.set_ylabel(f"{_year}")
    
    # GRID
    ax[_i].grid(axis='x', which='major', visible=False)
    ax[_i].grid(axis='y', which='major', visible=True)
    
    # SPINES
    # Can't forget that the secondary Y axis is based on a duplicate AXES, so it needs disabling spine too.
    sec_y.spines.right.set_visible(False)
    ax[_i].spines.top.set_visible(False)
    ax[_i].spines.right.set_visible(False)
    ax[_i].spines.left.set_visible(False)
    
    # FORMATTING Y AXIS TICK LABELS
    # See documentation on why the lambda function has two arguments and use only one
    # https://matplotlib.org/stable/api/ticker_api.html#matplotlib.ticker.FuncFormatter
    ax[_i].yaxis.set_major_formatter(ticker.FuncFormatter(lambda t, s: f"{t / 1000}"))

# TICK LABELS - ADD JUST LABELS TO THE BOTTOM AXES 
ax[-1].set_xticks(ticks=bar_xticks_major, labels=bar_xlabels)
    
# LEGEND (customized)
legend_elements = []
for _j, _categ in enumerate(categ):
    legend_elements.append(
        Patch(
            facecolor=sns.color_palette('tab20')[_j],
            edgecolor=sns.color_palette('tab20')[_j],
            label=_categ
        )
    )
fig.legend(handles=legend_elements, frameon=False, ncol=4, loc='lower center', bbox_to_anchor=(0.5, 0.03))

# TITLE
fig.suptitle(t="Monthly Expenses by Category - Selected Representative", y=0.92, fontsize=18)

# ADJUST SPACING BETWEEN PLOTS
plt.subplots_adjust(hspace=0.1)
    
plt.show()

I'll compare this Representative's budget with the annual average of the Brazilian Congress using the same categories.

In [ ]:
budget_complete = pd.concat([budget_2019, budget_2020, budget_2021, budget_2022]).copy()

In [ ]:
budget_complete.describe()

In [ ]:
# Applying new categories as well
budget_complete['txtClassification'] = budget_complete['txtDescricao'].map(descr_reclass).astype('category')

In [ ]:
# Since each state quota differs given the variations in air travel costs for each State,
# the amount spent on the Air Travel should be corrected by a factor relating each State to Sao Paulo (my home State).
# Chartered Flights will not receive such treatment.

quota = pd.read_csv('cota_parlamentar_por_UF.csv', delimiter=',', header=0, index_col=0,
                    dtype={'vlrCota': 'float'})
quota

In [ ]:
budget_complete.merge(right=quota, how='left', left_on='sgUF', right_index=True)

In [ ]:
# There are some NaN which are for Gov's, Minority's, Majority's, and Parties' Leaders, etc.
# These will be treated as if from SP (ratio=1).
budget_complete[budget_complete['sgUF'].isna()]['txNomeParlamentar'].unique()

In [ ]:
budget_complete['sgUF'].fillna(value='SP', inplace=True)

In [ ]:
budget_complete = budget_complete.merge(right=quota, how='left', left_on='sgUF', right_index=True)
budget_complete

In [ ]:
# Applying correction relative to SP quota
quota_sp = float(quota[quota.index == "SP"]['vlrCota'])
budget_complete['vlrLiquidoCorrigido'] = np.where(
    budget_complete['txtClassification'] == 'Air Travel',
    budget_complete['vlrLiquido'] * budget_complete['vlrCota'] / quota_sp,
    budget_complete['vlrLiquido']
)
budget_complete

In [ ]:
budget_complete.groupby(['numAno', 'numMes', 'txtClassification'])['vlrLiquidoCorrigido'].mean()

In [ ]:
# PLOT OF THE CONGRESS / HOUSE OF REPRESENTATIVES
# List of years in legislature
years = list(budget_complete['numAno'].unique())

# List of categories of expenses
categ = list(budget_complete['txtClassification'].unique().sort_values())

# SUBPLOTS
fig, ax = plt.subplots(figsize=(16, 3 * len(years)), sharex=False, sharey=True, ncols=1, nrows=len(years))

# CONSTANTS
bar_width = 0.6 / len(categ)
bar_xlim = (0, 13)
bar_y_interval = 2500
bar_y_max = max(budget_complete.groupby(['numAno', 'numMes', 'txtClassification'])['vlrLiquidoCorrigido'].mean())
# bar_y_max = 20_000
bar_ylim = (0, (bar_y_max // bar_y_interval + 1) * bar_y_interval)
bar_xticks_major = range(1, 13)
bar_xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# CREATE A BAR PLOT FOR EACH YEAR
for _i, _year in enumerate(years):
    
    # CREATE SHADING TO SEPARATE MONTHS
    for _k, _month in enumerate(bar_xlabels):
        if _k % 2 != 0:
            ax[_i].axvspan(
                xmin=(_k - 0.5),
                xmax=(_k + 0.5),
                ymin=0,
                ymax=1,
                color='lightgray',
                alpha=0.35
            )
    
    # CREATE DATA AND BARS FOR EACH CATEGORY
    for _j, _categ in enumerate(categ):
        bar_data = (
            budget_complete[
                (budget_complete['numAno'] == _year)
                & (budget_complete['txtClassification'] == _categ)
            ].groupby(['numMes']).mean()
        )
        
        _x = bar_data.index - (len(categ) / 2 - _j) * bar_width
        _y = bar_data['vlrLiquido']
        ax[_i].bar(x=_x, height=_y, width=bar_width, align='center', color=sns.color_palette('tab20')[_j])
    
    # LIMITS
    ax[_i].set_xlim(bar_xlim)
    ax[_i].set_ylim(bar_ylim)
    
    # TICK LABELS - REMOVE ALL FROM X AXIS
    ax[_i].set_xticks([])
    
    # AXIS LABELS
    ax[_i].set_ylabel("BRL 1k", labelpad=10)
    
    # SECONDARY Y AXIS
    # The creation of a secondary axis requires transformation functions. I just need the label...
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.secondary_yaxis.html
    sec_y = ax[_i].secondary_yaxis(location='right', functions=(lambda t: t, lambda t: t))
    sec_y.set_yticks([])
    sec_y.set_ylabel(f"{_year}")
    
    # GRID
    ax[_i].grid(axis='x', which='major', visible=False)
    ax[_i].grid(axis='y', which='major', visible=True)
    
    # SPINES
    # Can't forget that the secondary Y axis is based on a duplicate AXES, so it needs disabling spine too.
    sec_y.spines.right.set_visible(False)
    ax[_i].spines.top.set_visible(False)
    ax[_i].spines.right.set_visible(False)
    ax[_i].spines.left.set_visible(False)
    
    # FORMATTING Y AXIS TICK LABELS
    # See documentation on why the lambda function has two arguments and use only one
    # https://matplotlib.org/stable/api/ticker_api.html#matplotlib.ticker.FuncFormatter
    ax[_i].yaxis.set_major_formatter(ticker.FuncFormatter(lambda t, s: f"{t / 1000}"))

# TICK LABELS - ADD JUST LABELS TO THE BOTTOM AXES 
ax[-1].set_xticks(ticks=bar_xticks_major, labels=bar_xlabels)
    
# LEGEND (customized)
legend_elements = []
for _j, _categ in enumerate(categ):
    legend_elements.append(
        Patch(
            facecolor=sns.color_palette('tab20')[_j],
            edgecolor=sns.color_palette('tab20')[_j],
            label=_categ
        )
    )
fig.legend(handles=legend_elements, frameon=False, ncol=4, loc='lower center', bbox_to_anchor=(0.5, 0.01))

# TITLE
fig.suptitle(t="Monthly Mean Expenses by Category - All Representatives", y=0.92, fontsize=18)

# ADJUST SPACING BETWEEN PLOTS
plt.subplots_adjust(hspace=0.1)
    
plt.show()

In [ ]:
# Quick check on which parties spent more in chartered flights... No surprises here.
budget_chartered = budget_complete[budget_complete['txtClassification'] == 'Chartered Flights'][['txNomeParlamentar', 'sgUF', 'sgPartido', 'vlrLiquidoCorrigido']].copy()
budget_chartered.groupby('sgPartido').sum()['vlrLiquidoCorrigido'].apply(lambda t: f"R$ {t :,.2f}")

In [ ]:
# Checking how are the categories are distributed (needs improvement)
fig, ax = plt.subplots(figsize=(16, 9))

sns.boxplot(x=budget_complete['txtClassification'], y=budget_complete['vlrLiquidoCorrigido'], ax=ax)
ax.tick_params(axis='x', labelrotation=90)

In [ ]:
# Filtering off the party leaderships, etc.
(budget_complete[
    (budget_complete['numAno'] == 2019)
    & (~budget_complete['txNomeParlamentar'].str.startswith('LID'))
    & (budget_complete['nuLegislatura'] == 2019)
])

In [ ]:
# The number of representatives oscilate due to several reasons. The explanation is not on the scope of the project.
# Hence calculating the average monthly expense should be done manually.
repres_count = (budget_complete[
    (~budget_complete['txNomeParlamentar'].str.startswith('LID'))
    & (budget_complete['nuLegislatura'] == 2019)
].groupby(['numAno', 'numMes'])['txNomeParlamentar'].nunique())

repres_expenses = (budget_complete[
    (~budget_complete['txNomeParlamentar'].str.startswith('LID'))
    & (budget_complete['nuLegislatura'] == 2019)
].groupby(['numAno', 'numMes'])['vlrLiquidoCorrigido'].sum())

repres_avg = pd.concat([repres_count, repres_expenses], axis=1).reset_index()

repres_avg

In [ ]:
quota[quota.index == "SP"]['vlrCota']

In [ ]:
# Calculating average monthly expense and percentage of the maximum stablished budget.
repres_avg['vlrMedioMensal'] = repres_avg['vlrLiquidoCorrigido'] / repres_avg['txNomeParlamentar']
repres_avg['percentTeto'] = round(repres_avg['vlrMedioMensal'] / float(quota[quota.index == "SP"]['vlrCota']) * 100, 1)
repres_avg

In [ ]:
budget_totals = budget.groupby(['numAno', 'numMes'])['vlrLiquido'].sum().reset_index()
budget_totals

In [ ]:
# PLOTTING SELECTED REPRESENTATIVE MONTHLY TOTAL IN COMPARISON TO THE HOUSE'S MONTHLY AVERAGE
# "Constants"
years = list(repres_avg['numAno'].unique())
lin_xlim = (0.5, 12.5)
lin_xticks = range(1, 13)
lin_xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
lin_y_interval = 12500
lin_y_max = (max(repres_avg['vlrMedioMensal']) // lin_y_interval + 1) * lin_y_interval
lin_ylim = (0, lin_y_max + 10000)
lin_yticks = np.linspace(0, lin_y_max, int(lin_y_max // lin_y_interval))
monthly_limit = float(quota[quota.index == "SP"]['vlrCota'])


fig, ax = plt.subplots(figsize=(16, 9), nrows=len(years), ncols=1, sharex=True, sharey=True)

for _i, _year in enumerate(years):
    line_data = repres_avg[repres_avg['numAno'] == _year]
    scatter_data = budget_totals[budget_totals['numAno'] == _year]
    ax[_i].axhline(y=monthly_limit, xmin=0, xmax=1, color='grey', linestyle='--', alpha=0.9)
    ax[_i].plot(line_data['numMes'], line_data['vlrMedioMensal'], color='dodgerblue', linewidth=2.0)
    ax[_i].scatter(scatter_data['numMes'], scatter_data['vlrLiquido'], color='darkorange')
    
    sec_y = ax[_i].secondary_yaxis(location='right', functions=(lambda t: t, lambda t: t))
    sec_y.set_yticks([])
    sec_y.set_ylabel(f"{_year}")
    
    ax[_i].set_ylabel("BRL 1k", labelpad=10)
    ax[_i].yaxis.set_major_formatter(ticker.FuncFormatter(lambda t, s: f"{t / 1000}"))
    
    ax[_i].grid(axis='x', which='major', visible=True, alpha=0.6)
    ax[_i].grid(axis='y', which='major', visible=True, alpha=0.6)
    
    sec_y.spines.right.set_visible(False)
    ax[_i].spines.top.set_visible(False)
    ax[_i].spines.right.set_visible(False)
    ax[_i].spines.left.set_visible(False)
    
ax[-1].set_xlim(lin_xlim)
ax[-1].set_ylim(lin_ylim)   
ax[-1].set_xticks(ticks=lin_xticks, labels=lin_xlabels)
ax[-1].set_yticks(ticks=lin_yticks)

# LEGEND (customized)
legend_elements = []
legend_elements.append(Patch(facecolor='dodgerblue', edgecolor='dodgerblue', label='House Average'))
legend_elements.append(
    Line2D([], [], color='darkorange', marker='o', linestyle='', markersize=10,
           label='Selected Representative Total')
)
legend_elements.append(
    Line2D([], [], color='grey', marker='', linestyle='--', alpha=0.9,
           label='Limit Monthly Budget (São Paulo)')
)
fig.legend(handles=legend_elements, frameon=False, ncol=4, loc='lower center', bbox_to_anchor=(0.5, 0.01))

fig.suptitle(t="Monthly Mean of Representatives & Monthly Total for Selected Representative", y=0.92, fontsize=18)

plt.subplots_adjust(hspace=0.1)

plt.show()

In [ ]:
# Since the house spend, on average, a big part of its budget on Consulting, I want to check how my Representative
# spends on this category. Checking the profile of the suppliers.
budget_details = budget[budget['txtClassification'] == "Consulting"].groupby(['txtFornecedor'])['vlrLiquido'].sum().sort_values(ascending=False).reset_index()
budget_details

In [ ]:
# Anonymizing the suppliers for publication
supplier_unnamed = dict()
for _t, _supp in enumerate(budget_details['txtFornecedor']):
    supplier_unnamed[_supp] = "Supplier " + chr(_t + 65)

supplier_unnamed

In [ ]:
budget_details['txtSupplierUnnamed'] = budget_details['txtFornecedor'].apply(lambda t: supplier_unnamed[t])
budget_details['vlrLiquidoAcum'] = budget_details['vlrLiquido'].cumsum()
budget_details['vlrLiquidoAcumPerc'] = budget_details['vlrLiquidoAcum'] / max(budget_details['vlrLiquidoAcum']) * 100
budget_details

In [ ]:
budget_details[['txtSupplierUnnamed', 'vlrLiquido', 'vlrLiquidoAcum', 'vlrLiquidoAcumPerc']]